In [3]:
from glob import  glob
from pathlib import Path

import numpy

from become_yukarin.config.config import create_from_json as create_config
from become_yukarin.config.sr_config import create_from_json as create_sr_config
from become_yukarin.dataset.dataset import AcousticFeatureProcess
from become_yukarin.dataset.dataset import WaveFileLoadProcess
from become_yukarin.param import Param
from become_yukarin import SuperResolution
from become_yukarin import AcousticConverter

from IPython.display import Audio

# loading model1
model_path = Path('./dat/model/yukari_1st_dataset/tori_raw_predictor_150000.npz')
config_path = Path('./dat/model/yukari_1st/config.json')
config = create_config(config_path)
voice_changer = AcousticConverter(config, model_path, gpu=0)

# loading model2
model_path = Path('./dat/model/yukari_2nd/predictor_120000.npz')
config_path = Path('./dat/model/yukari_2nd/config.json')
sr_config = create_sr_config(config_path)
super_resolution = SuperResolution(sr_config, model_path, gpu=0)

In [13]:
import pyaudio
import wave
import soundfile as sf
import os
import time

for i in range(100):
    print(i)
    # recoding process
    start = time.time()
    rec_time = 5 # 録音時間
    file_path = "./test_data/output.wav" # 音声を保存するファイル名
    fmt = pyaudio.paInt16 # 音声のフォーマット
    ch = 1 # チャンネル数
    sampling_rate = 22050 # サンプリング周波数
    chunk = 2**11 # チャンク（データ点数）
    audio = pyaudio.PyAudio()
    index = 26 # 録音デバイスのインデックス番号

    print("recoding start ...")

    stream = audio.open(format=fmt, channels=ch, rate=sampling_rate, input=True,
                   input_device_index = index, frames_per_buffer=chunk)
    # 録音処理
    frames = []
    for i in range(0, int(sampling_rate / chunk * rec_time)):
        data = stream.read(chunk)
        frames.append(data)

    print("recoding end...")

    # 録音終了処理
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # 録音データをファイルに保存
    import wave
    wav = wave.open(file_path, 'wb')
    wav.setnchannels(ch)
    wav.setsampwidth(audio.get_sample_size(fmt))
    wav.setframerate(sampling_rate)
    wav.writeframes(b''.join(frames))
    wav.close()

    # test voice conversion
    p = './test_data/output.wav'
    wave_in = voice_changer._wave_process(str(p))
    f_in = voice_changer._feature_process(wave_in)
    f_low = voice_changer.convert_to_feature(f_in)
    s_high = super_resolution.convert(f_low.spectrogram.astype(numpy.float32))

    # make audio
    rate = sr_config.dataset.param.voice_param.sample_rate
    wave = super_resolution(s_high, acoustic_feature=f_low, sampling_rate=rate)
    sf.write('./test_data/vc.wav', wave.wave, rate)
    elapsed_time = time.time() - start
    os.system("aplay --quiet './test_data/vc.wav' ")
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

0
recoding start ...
recoding end...
elapsed_time:7.697715520858765[sec]
1
recoding start ...
recoding end...
elapsed_time:7.634891510009766[sec]
2
recoding start ...
recoding end...
elapsed_time:7.640070915222168[sec]
3
recoding start ...
recoding end...
elapsed_time:7.66875147819519[sec]
4
recoding start ...
recoding end...
elapsed_time:7.664248943328857[sec]
5
recoding start ...
recoding end...
elapsed_time:7.665453910827637[sec]
6
recoding start ...
recoding end...
elapsed_time:7.622462511062622[sec]
7
recoding start ...
recoding end...
elapsed_time:7.701231956481934[sec]
8
recoding start ...
recoding end...
elapsed_time:7.756594181060791[sec]
9
recoding start ...
recoding end...
elapsed_time:7.7513227462768555[sec]
10
recoding start ...
recoding end...
elapsed_time:7.605807304382324[sec]
11
recoding start ...
recoding end...
elapsed_time:7.667892694473267[sec]
12
recoding start ...
recoding end...
elapsed_time:7.6338183879852295[sec]
13
recoding start ...
recoding end...
elapsed_t

In [12]:
wave

Wave(wave=array([-0.00014989, -0.0001286 , -0.00012303, ..., -0.00199063,
       -0.00174656, -0.00147358]), sampling_rate=22050)

In [14]:
wave.sampling_rate

22050

In [11]:
rate

22050